# Jupyter notebook for AI image compression
## Importing packages and setting parameters

In [39]:
import math
import torch
from torchvision import transforms
import glob
from PIL import Image
import ntpath
import os
from pathlib import Path
import timeit
from datetime import datetime
from numpy import mean

In [33]:
from compressai.zoo import (mbt2018_mean, mbt2018, cheng2020_anchor, cheng2020_attn)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
metric = 'mse'  # only pre-trained model for mse are available for now

current_path = Path('.').resolve()
print("Current path: ",str(current_path))
source_path = str(current_path)+'/IMAGES/PNG_IMAGES'
source_path.replace(os.sep,ntpath.sep)
print("Source: ",source_path)

Current path:  /home/filippo/DataspellProjects/ImageCompressionAI
Source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/PNG_IMAGES


## Loading some pretrained models

In [34]:
networks = {
    'mbt2018_mean_1' : mbt2018_mean(quality=1, pretrained=True).eval().to(device),
    'mbt2018_mean_3' : mbt2018_mean(quality=4, pretrained=True).eval().to(device),
    'mbt2018_mean_4' : mbt2018_mean(quality=5, pretrained=True).eval().to(device),
    'mbt2018_mean_5' : mbt2018_mean(quality=6, pretrained=True).eval().to(device),
    'mbt2018_1': mbt2018(quality=1, pretrained=True).eval().to(device),
    'mbt2018_2': mbt2018(quality=2, pretrained=True).eval().to(device),
    'mbt2018_3': mbt2018(quality=4, pretrained=True).eval().to(device),
    'mbt2018_4': mbt2018(quality=5, pretrained=True).eval().to(device),
    'mbt2018_5': mbt2018(quality=6, pretrained=True).eval().to(device),
    'cheng2020-anchor_1': cheng2020_anchor(quality=1, pretrained=True).eval().to(device),
    'cheng2020-anchor_2': cheng2020_anchor(quality=2, pretrained=True).eval().to(device),
    'cheng2020-anchor_3': cheng2020_anchor(quality=4, pretrained=True).eval().to(device),
    'cheng2020-anchor_4': cheng2020_anchor(quality=5, pretrained=True).eval().to(device),
    'cheng2020-anchor_5': cheng2020_anchor(quality=6, pretrained=True).eval().to(device),
    'cheng2020_attn_1' : cheng2020_attn(quality=1, pretrained=True).eval().to(device),
    'cheng2020_attn_2' : cheng2020_attn(quality=2, pretrained=True).eval().to(device),
    'cheng2020_attn_3' : cheng2020_attn(quality=4, pretrained=True).eval().to(device),
    'cheng2020_attn_4' : cheng2020_attn(quality=5, pretrained=True).eval().to(device),
    'cheng2020_attn_5' : cheng2020_attn(quality=6, pretrained=True).eval().to(device)
}

## Compression with each network

In [36]:
destination_path = str(current_path)+'/IMAGES/NET'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/NET
Destination directory has been emptied


In [40]:
for name, net in networks.items():
    destination_path_net = str(current_path)+'/IMAGES/NET/'+str(name)
    destination_path_net.replace(os.sep,ntpath.sep)
    print("Destination: ",destination_path_net)
    if os.path.isdir(destination_path_net) & os.path.exists(destination_path_net):
        try:
            files = os.listdir(destination_path_net)
            for file in files:
                file_path = os.path.join(destination_path_net, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print("Destination directory has been emptied")
        except OSError:
            print("Error occurred while deleting files in the destination directory")
    else:
        os.mkdir(destination_path_net)
    compression_times = []
    bpp_per_image = []
    log = open(str(destination_path_net)+'/log.txt', 'w')
    log.writelines([str(datetime.now()),'\n'])
    count = 0
    
    for file in glob.glob(str(source_path)+'/*.png'):
        img = Image.img = Image.open(file).convert('RGB')
        x = transforms.ToTensor()(img).unsqueeze(0).to(device)
        outputs = {}
        with torch.no_grad():
                starttime = timeit.default_timer()
                rv = net(x)
                endtime = timeit.default_timer()
                rv['x_hat'].clamp_(0, 1)
                outputs[Path(file).stem] = rv
        execution_time = endtime-starttime
        compression_times.append(execution_time)
        num_pixels = rv['x_hat'].size()[0] * rv['x_hat'].size()[2] * rv['x_hat'].size()[3]
        file_size = sum(torch.log(likelihoods).sum() for likelihoods in rv['likelihoods'].values()).item()
        bits_per_pixel = file_size / (-math.log(2) * num_pixels)
        bpp_per_image.append(bits_per_pixel)
        log_string = 'Image:' + str(Path(file).stem) + '.png\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(-file_size) + '\tWidth:' + str(rv['x_hat'].size()[2]) + '\tHeight:' + str(rv['x_hat'].size()[3]) + '\tPixels:' + str(num_pixels) + '\tCompressionTime:' + str(compression_times[count])
        log.writelines([log_string,'\n'])
        print(log_string)
        transforms.ToPILImage()(rv['x_hat'].squeeze()).save(str(destination_path_net+'/'+str(Path(file).stem)+'.png'),'png')
        img.close()
        count +=1

    log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_per_image)) + ' in ' + str(mean(compression_times)) + ' seconds')])
    print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_per_image)) + ' in ' + str(mean(compression_times)) + ' seconds' + '\n')
    log.close()

Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/NET/mbt2018_mean_1
Destination directory has been emptied
Image:IMG0001.png	bpp:0.08713531102928102	Size:379988.8125	Width:2048	Height:3072	Pixels:6291456	CompressionTime:4.900609880999582
Image:IMG0002.png	bpp:0.06969518817428054	Size:303934.09375	Width:2048	Height:3072	Pixels:6291456	CompressionTime:4.841425550000167
Image:IMG0003.png	bpp:0.03126827682549592	Size:136357.984375	Width:2048	Height:3072	Pixels:6291456	CompressionTime:5.020367909000015
Image:IMG0004.png	bpp:0.052253241586586535	Size:227871.421875	Width:3072	Height:2048	Pixels:6291456	CompressionTime:4.881129489999694
Image:IMG0005.png	bpp:0.08384787698992599	Size:365652.625	Width:2048	Height:3072	Pixels:6291456	CompressionTime:4.969841408999855
Image:IMG0006.png	bpp:0.06430644860885237	Size:280434.3125	Width:2048	Height:3072	Pixels:6291456	CompressionTime:4.8388661620001585
Image:IMG0007.png	bpp:0.041580987174283095	Size:181330.734375	Width:2048	Heigh